# Setting up environment and Loading data from json file into dataframe

In [1]:
# Finally, setup our Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
spark

In [3]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns
from pyspark.sql.functions import *
from pyspark.sql.functions import col, trim, length, to_date, year, month, collect_list, max
from pyspark.sql import functions as F 

In [38]:
# Load the data
df_lighter_books = spark.read.json("lighter_books.json")
df_lighter_authors = spark.read.json("lighter_authors.json")

## General Data Cleaning/Preprocessing

In [78]:
# Drop the duplicates
df_lighter_books = df_lighter_books.dropDuplicates()
df_lighter_authors = df_lighter_authors.dropDuplicates()

# [RQ1] General Exploratory Data Analysis (EDA)

In [79]:
# Print the schema of the dataframe
df_lighter_books.printSchema()

root
 |-- asin: string (nullable = true)
 |-- author_id: long (nullable = true)
 |-- author_name: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- role: string (nullable = true)
 |-- average_rating: double (nullable = true)
 |-- description: string (nullable = true)
 |-- edition_information: string (nullable = true)
 |-- format: string (nullable = true)
 |-- id: long (nullable = true)
 |-- image_url: string (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- language: string (nullable = true)
 |-- num_pages: long (nullable = true)
 |-- original_publication_date: string (nullable = true)
 |-- publication_date: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- rating_dist: string (nullable = true)
 |-- ratings_count: long (nullable = true)
 |-- series_id: str

In [80]:
# Print the schema of the dataframe
df_lighter_authors.printSchema()

root
 |-- about: string (nullable = true)
 |-- average_rating: double (nullable = true)
 |-- book_ids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fans_count: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- id: long (nullable = true)
 |-- image_url: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ratings_count: long (nullable = true)
 |-- text_reviews_count: long (nullable = true)
 |-- work_ids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- works_count: long (nullable = true)



In [81]:
# Print the first 5 rows of the dataframe
df_lighter_books.show(5)

+----+---------+---------------+--------------------+--------------+--------------------+-------------------+---------+-----+--------------------+----------+-------------+--------+---------+-------------------------+----------------+-----------------+--------------------+-------------+---------+-----------+---------------+---------------+------------------+--------------------+-------+
|asin|author_id|    author_name|             authors|average_rating|         description|edition_information|   format|   id|           image_url|      isbn|       isbn13|language|num_pages|original_publication_date|publication_date|        publisher|         rating_dist|ratings_count|series_id|series_name|series_position|        shelves|text_reviews_count|               title|work_id|
+----+---------+---------------+--------------------+--------------+--------------------+-------------------+---------+-----+--------------------+----------+-------------+--------+---------+-------------------------+------

In [82]:
# Print the first 5 rows of the dataframe
df_lighter_authors.show(5)

+--------------------+--------------+--------------------+----------+------+--------+--------------------+--------------------+-------------+------------------+--------------------+-----------+
|               about|average_rating|            book_ids|fans_count|gender|      id|           image_url|                name|ratings_count|text_reviews_count|            work_ids|works_count|
+--------------------+--------------+--------------------+----------+------+--------+--------------------+--------------------+-------------+------------------+--------------------+-----------+
|<i>Librarian Note...|          2.39|[37639181, 351301...|         1|      |16850813|https://s.gr-asse...|          Adil Aijaz|           18|                 2|[59226555, 564512...|          4|
|American ambassad...|          3.06|[290618, 735481, ...|         0|  male|   84378|https://s.gr-asse...|Thomas Patrick Me...|           18|                 5|[281951, 721665, ...|          9|
|Robert Mailer And...|        

In [83]:
# Print the head of dataframe
df_lighter_books.head()

Row(asin='', author_id=1077326, author_name='J.K. Rowling', authors=[Row(id='1077326', name='J.K. Rowling', role=''), Row(id='2927', name='Mary GrandPré', role='Illustrator')], average_rating=4.5, description='There is a door at the end of a silent corridor. And it’s haunting Harry Pottter’s dreams. Why else would he be waking in the middle of the night, screaming in terror?<br /><br />Harry has a lot on his mind for this, his fifth year at Hogwarts: a Defense Against the Dark Arts teacher with a personality like poisoned honey; a big surprise on the Gryffindor Quidditch team; and the looming terror of the Ordinary Wizarding Level exams. But all these things pale next to the growing threat of He-Who-Must-Not-Be-Named - a threat that neither the magical government nor the authorities at Hogwarts can stop.<br /><br />As the grasp of darkness tightens, Harry must discover the true depth and strength of his friends, the importance of boundless loyalty, and the shocking price of unbearable 

In [84]:
# Print the head of dataframe
df_lighter_authors.head()

Row(about='Douglas Noël Adams was an English author, comic radio dramatist, and musician. He is best known as the author of the <i>\n  <a href="https://www.goodreads.com/book/show/11.Hitchhiker_s_Guide_to_the_Galaxy" title="Hitchhiker\'s Guide to the Galaxy" rel="nofollow noopener">Hitchhiker\'s Guide to the Galaxy</a>\n</i> series. Hitchhiker\'s began on radio, and developed into a "trilogy" of five books (which sold more than fifteen million copies during his lifetime) as well as a television series, a comic book series, a computer game, and a feature film that was completed after Adams\' death. The series has also been adapted for live theatre using various scripts; the earliest such productions used material newly written by Adams. He was known to some fans as Bop Ad (after his illegible signature), or by his initials "DNA".<br /><br />In addition to <i>The Hitchhiker\'s Guide to the Galaxy</i>, Douglas Adams wrote or co-wrote three stories of the science fiction television series 

In [85]:
# summary statistics of the dataframe
df_lighter_books.describe().show()

+-------+--------+-----------------+-------------------+------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------+--------------------+--------+------------------+-------------------------+------------------+----------------------+--------------------+------------------+----------------+-----------------------------------+-----------------+------------------+-------+--------------------+
|summary|    asin|        author_id|        author_name|    average_rating|         description| edition_information|            format|                  id|           image_url|    isbn|              isbn13|language|         num_pages|original_publication_date|  publication_date|             publisher|         rating_dist|     ratings_count|       series_id|                        series_name|  series_position|text_reviews_count|  title|             work_id|
+-------+--------+-----------------+-------------------+----------------

In [86]:
# summary statistics of the dataframe
df_lighter_authors.describe().show()

+-------+--------------------+------------------+------------------+---------+------------------+--------------------+-------------------+-----------------+------------------+------------------+
|summary|               about|    average_rating|        fans_count|   gender|                id|           image_url|               name|    ratings_count|text_reviews_count|       works_count|
+-------+--------------------+------------------+------------------+---------+------------------+--------------------+-------------------+-----------------+------------------+------------------+
|  count|              351767|            351767|            351767|   351767|            351767|              351767|             351767|           351767|            351767|            351767|
|   mean|   1752.326923076923|3.6511943132812488|111.61573143586521|     NULL|7751861.1911975825|                NULL|           Infinity|4770.586308550831| 330.9932426862099|25.937137366495435|
| stddev|  437.4607532493

In [87]:
# columns of the dataframe
df_lighter_books.columns

['asin',
 'author_id',
 'author_name',
 'authors',
 'average_rating',
 'description',
 'edition_information',
 'format',
 'id',
 'image_url',
 'isbn',
 'isbn13',
 'language',
 'num_pages',
 'original_publication_date',
 'publication_date',
 'publisher',
 'rating_dist',
 'ratings_count',
 'series_id',
 'series_name',
 'series_position',
 'shelves',
 'text_reviews_count',
 'title',
 'work_id']

In [88]:
# columns of the dataframe
df_lighter_authors.columns

['about',
 'average_rating',
 'book_ids',
 'fans_count',
 'gender',
 'id',
 'image_url',
 'name',
 'ratings_count',
 'text_reviews_count',
 'work_ids',
 'works_count']

In [89]:
# Count the number of rows in the dataframe
df_lighter_books.count()

7027431

In [90]:
# Count the number of rows in the dataframe
df_lighter_authors.count()

351767

In [91]:
# Count the number of distinct rows in the dataframe
df_lighter_books.distinct().count()

7027431

In [92]:
# Count the number of distinct rows in the dataframe
df_lighter_authors.distinct().count()

351767

# [RQ2]

# [RQ3]

In [5]:
# Print the schema of the dataframe
df_lighter_books.printSchema()

root
 |-- asin: string (nullable = true)
 |-- author_id: long (nullable = true)
 |-- author_name: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- role: string (nullable = true)
 |-- average_rating: double (nullable = true)
 |-- description: string (nullable = true)
 |-- edition_information: string (nullable = true)
 |-- format: string (nullable = true)
 |-- id: long (nullable = true)
 |-- image_url: string (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- language: string (nullable = true)
 |-- num_pages: long (nullable = true)
 |-- original_publication_date: string (nullable = true)
 |-- publication_date: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- rating_dist: string (nullable = true)
 |-- ratings_count: long (nullable = true)
 |-- series_id: str

## Data Cleaning/Preprocessing

In [6]:
df_lighter_books = df_lighter_books.filter(trim(col('original_publication_date')) != '') # removes rows with empty original_publication_date
df_lighter_books = df_lighter_books.filter(trim(col('num_pages')) != '') # removes rows with empty num_pages
df_lighter_books = df_lighter_books.filter(trim(col('title')) != '') # removes rows with empty title

In [7]:
df_lighter_books = df_lighter_books.dropna(subset=['original_publication_date', 'num_pages','title']) # removes rows with null values in original_publication_date, num_pages, and title

In [8]:
# Define the regular expression for date
date_regex = r"^(19\d\d|200[0-2]|20(0[3-9]|1\d|2[0-3]))-(0[1-9]|1[012])-(0[1-9]|1[0-9]|2[0-9]|3[01])$"

# Validate the original_publication_date column
df_lighter_books = df_lighter_books.withColumn(
    "is_valid_date",
    F.when(F.col("original_publication_date").rlike(date_regex), True).otherwise(False)
)

# Filter the DataFrame to keep only rows with valid dates
df_lighter_books = df_lighter_books.filter(F.col("is_valid_date") == True)

# Drop the is_valid_date column
df_lighter_books = df_lighter_books.drop("is_valid_date")

In [9]:
# Attempt to convert original_publication_date to a date
df_lighter_books = df_lighter_books.withColumn("converted_date", to_date(col("original_publication_date"), 'yyyy-MM-dd'))

# Filter out rows where the conversion resulted in null
df_lighter_books = df_lighter_books.filter(col("converted_date").isNotNull())

# Drop the intermediate converted_date column
df_lighter_books = df_lighter_books.drop("converted_date")

In [10]:
df_lighter_books = df_lighter_books.withColumn("title", trim(df_lighter_books.title))  # removes leading and trailing white spaces

In [11]:
# Filter for valid num_pages
df_lighter_books = df_lighter_books.filter(col('num_pages') > 0)

# Select the relevant columns
df_lighter_books = df_lighter_books.select('original_publication_date', 'num_pages', 'title')

## EDA

In [12]:
df_lighter_books.count() # count the number of rows

2390718

In [13]:
df_lighter_books.show(5) # show the first 5 rows

+-------------------------+---------+--------------------+
|original_publication_date|num_pages|               title|
+-------------------------+---------+--------------------+
|               2003-06-21|      870|Harry Potter and ...|
|               1997-06-26|      309|Harry Potter and ...|
|               1998-07-02|      352|Harry Potter and ...|
|               1999-07-08|      435|Harry Potter and ...|
|               2000-07-08|      734|Harry Potter and ...|
+-------------------------+---------+--------------------+
only showing top 5 rows



In [15]:
df_lighter_books = df_lighter_books.withColumn('year', year('original_publication_date')) # add a year column
df_lighter_books = df_lighter_books.withColumn('month', month('original_publication_date')) # add a month column

In [23]:
df_lighter_books.show(5) # show the first 5 rows

+-------------------------+---------+--------------------+----+-----+
|original_publication_date|num_pages|               title|year|month|
+-------------------------+---------+--------------------+----+-----+
|               2003-06-21|      870|Harry Potter and ...|2003|    6|
|               1997-06-26|      309|Harry Potter and ...|1997|    6|
|               1998-07-02|      352|Harry Potter and ...|1998|    7|
|               1999-07-08|      435|Harry Potter and ...|1999|    7|
|               2000-07-08|      734|Harry Potter and ...|2000|    7|
+-------------------------+---------+--------------------+----+-----+
only showing top 5 rows



In [59]:
df_lighter_books.select("year").distinct().count() # count the number of distinct years

124

In [60]:
max_year = df_lighter_books.agg(max("year")).first()[0] # find the maximum year
print(max_year)

2023


In [61]:
min_year = df_lighter_books.agg(min("year")).first()[0] # find the minimum year
print(min_year)

1900


### yearly_stats function to get yearly stats for each year in the df_lighter_books dataframe

In [39]:
def yearly_stats(year):
    # Filter books for given year
    df_year = df_lighter_books.filter(df_lighter_books['year'] == year)

    # Number of books published
    num_books = df_year.count()

    # Total number of pages
    total_pages = df_year.agg(F.sum('num_pages')).first()[0]

    # Most prolific month
    prolific_month = df_year.groupBy('month').count().orderBy(F.desc('count')).first()[0]

    # Longest book
    longest_book = df_year.orderBy(F.desc('num_pages')).first()['title']

    return (year, num_books, total_pages, prolific_month, longest_book)

In [41]:
# Get list of unique years

years = df_lighter_books.select('year').toPandas()['year'].unique().tolist()

In [42]:
years.sort() # sort the list (not necessary)

In [50]:
data_first_10 = [yearly_stats(year) for year in years[:10]] # get the yearly stats for the first 10 years

In [51]:
data_last_10 = [yearly_stats(year) for year in years[-10:]] # get the yearly stats for the last 10 years

In [52]:
data_first_10 # print the yearly stats for the first 10 years

[(1900, 1747, 509501, 1, 'Complete Works of Joseph Conrad'),
 (1901, 802, 351965, 1, 'NKJV Study Bible'),
 (1902, 882, 696460, 1, 'Holy Bible: NLT - New Living Translation'),
 (1903, 733, 183484, 1, 'The Life of William Ewart Gladstone - Vol. I'),
 (1904, 628, 162193, 1, 'The Life Recovery Bible NLT'),
 (1905, 1119, 282252, 1, 'Dictionary of the Bible'),
 (1906, 518, 145632, 1, "Moody's Magazine Vol 1 - 20"),
 (1907, 558, 137774, 1, 'Arsenio Lupin, Caballero Ladrón'),
 (1908, 430, 97105, 1, 'Anne of Green Gables--The Complete Collection'),
 (1909, 1041, 298330, 1, 'The Works of Rudyard Kipling, 10 Vols')]

In [53]:
data_last_10   # print the yearly stats for the last 10 years

[(2014,
  177582,
  42919290,
  1,
  'A Most Unlikely Countess (To Love a Wildcat, #2)'),
 (2015,
  53821,
  14013926,
  1,
  'Revel for the American Nation: A History of the United States, Combined Volume -- Access Card'),
 (2016, 2299, 708061, 1, 'Homestuck'),
 (2017, 491, 150653, 2, 'The Starfarers Quartet'),
 (2018, 192, 72498, 1, '地海六部曲'),
 (2019, 118, 34813, 8, 'Nouvelles Les'),
 (2020, 83, 27477, 1, 'The Complete Ripley Novels (Ripley, #1-5)'),
 (2021,
  17,
  8088,
  8,
  "The Navigator's Children (The Last King of Osten Ard, #3)"),
 (2022,
  4,
  1280,
  10,
  'Highland Ever After (The Montgomerys and Armstrongs, #3)'),
 (2023, 1, 463, 9, 'Apocalypse')]

In [56]:
data_first_10 = pd.DataFrame(data_first_10, columns =['year', 'num_books', 'total_pages', 'prolific_month', 'longest_book']) # convert the list to a pandas dataframe

In [57]:
data_first_10   # print the dataframe

,year,num_books,total_pages,prolific_month,longest_book
0,1900,1747,509501,1,Complete Works of Joseph Conrad
1,1901,802,351965,1,NKJV Study Bible
2,1902,882,696460,1,Holy Bible: NLT - New Living Translation
3,1903,733,183484,1,The Life of William Ewart Gladstone - Vol. I
4,1904,628,162193,1,The Life Recovery Bible NLT
5,1905,1119,282252,1,Dictionary of the Bible
6,1906,518,145632,1,Moody's Magazine Vol 1 - 20
7,1907,558,137774,1,"Arsenio Lupin, Caballero Ladrón"
8,1908,430,97105,1,Anne of Green Gables--The Complete Collection
9,1909,1041,298330,1,"The Works of Rudyard Kipling, 10 Vols"


In [54]:
data_last_10 = pd.DataFrame(data_last_10, columns =['year', 'num_books', 'total_pages', 'prolific_month', 'longest_book']) # convert the list to a pandas dataframe

In [55]:
data_last_10  # print the dataframe

,year,num_books,total_pages,prolific_month,longest_book
0,2014,177582,42919290,1,"A Most Unlikely Countess (To Love a Wildcat, #2)"
1,2015,53821,14013926,1,Revel for the American Nation: A History of th...
2,2016,2299,708061,1,Homestuck
3,2017,491,150653,2,The Starfarers Quartet
4,2018,192,72498,1,地海六部曲
5,2019,118,34813,8,Nouvelles Les
6,2020,83,27477,1,"The Complete Ripley Novels (Ripley, #1-5)"
7,2021,17,8088,8,The Navigator's Children (The Last King of Ost...
8,2022,4,1280,10,Highland Ever After (The Montgomerys and Armst...
9,2023,1,463,9,Apocalypse


In [58]:
data_first_10.to_json(r'C:\Users\ambar\OneDrive\Desktop\Notes\ADM\ADM-HW2-main\RQ3\data_first_10.json', orient='records') # save the dataframe as a json file
data_last_10.to_json(r'C:\Users\ambar\OneDrive\Desktop\Notes\ADM\ADM-HW2-main\RQ3\data_last_10.json', orient='records') # save the dataframe as a json file

# [RQ4]

## Data Cleaning/Preprocessing

In [39]:
# Remove whitespace from the 'name' field
df_lighter_authors = df_lighter_authors.withColumn('name', trim(col('name')))

# Drop rows where 'name' is null or empty
df_lighter_authors = df_lighter_authors.filter((col('name') != "") & (col('name').isNotNull()))

# Drop rows where 'id' is null
df_lighter_authors = df_lighter_authors.filter(col('id').isNotNull())

## EDA

In [40]:
duplicate_authors = df_lighter_authors.groupBy("name").agg(count("id").alias("num_ids")).filter("num_ids > 1") # find the duplicate authors

In [41]:
duplicate_authors.count() # count the number of duplicate authors

37

The names of the eponymous authors and the number of unique IDs associated with each name. Hence, the answer to the question is _false_.

In [42]:
duplicate_authors.show(50) # show the duplicate authors

+--------------------+-------+
|                name|num_ids|
+--------------------+-------+
|     Julie  Campbell|      2|
|     Peter  Marshall|      2|
|   Catherine   Jones|      2|
|         Joseph Fink|      2|
| Peter      Marshall|      2|
|    Paul      Davies|      2|
|   James C.L. Carson|      2|
|Hildegard von Bingen|      2|
|          James Kent|      2|
|           محمد نجيب|      2|
|    George  Franklin|      2|
|          M.K. Graff|      2|
|      Martin    Shaw|      2|
|       Erin  Bedford|      2|
|William Messner-L...|      2|
|         David Yates|      2|
|         Paul Graham|      2|
|       Peter  Davies|      2|
|Katherine Mercuri...|      2|
|       Dimitar Dimov|      2|
|       David  Nelson|      2|
|        Q. Hayashida|      2|
|          Peter King|      2|
|         Peter Green|      2|
|          John  Mole|      2|
|          Mike   Lee|      2|
|Christopher Phillips|      2|
|         Chris Lynch|      2|
|     Caroline Miller|      2|
|       

## Data Cleaning/Preprocessing

In [85]:
df_lighter_books = df_lighter_books.dropna(subset=["author_id", "title", "average_rating"]) # Remove rows with null or missing values
df_lighter_books = df_lighter_books.filter(trim(col('title')) != '') # removes rows with empty title
df_lighter_books = df_lighter_books.withColumn("title", trim(col("title"))) # Trim whitespace from the title column

## EDA

A function that, given a list of author_id, outputs a dictionary where each author_id is a key, and the related value is a list with the names of all the books the author has written.

In [86]:
def get_author_books(author_ids):

    # Filter df_authors on the provided author_ids
    df_author_books = df_lighter_books.filter(df_lighter_books['author_id'].isin(author_ids))
    
    # Group by author id and collect all book titles
    df_author_books = df_author_books.groupBy("author_id").agg(collect_list("title").alias("books"))
    
    # Convert to Python dictionary
    author_books_dict = {row['author_id']: row['books'] for row in df_author_books.collect()}
    
    return author_books_dict

In [87]:
# List of author IDs
author_ids = [4]

# Use the function
author_books_dict = get_author_books(author_ids)

# Print the result
print(author_books_dict)

{4: ["The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1)", "The Ultimate Hitchhiker's Guide: Five Complete Novels and One Story (Hitchhiker's Guide to the Galaxy, #1-5)", "The Ultimate Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1-5)", "The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1)", "The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1)", "The Hitchhiker's Guide to the Galaxy: Quandary Phase (Hitchhiker's Guide: Radio Play, #4)", "The Ultimate Hitchhiker's Guide (Hitchhiker's Guide to the Galaxy, #1-5)", "The Hitchhiker's Guide to the Galaxy: Quintessential Phase (Hitchhiker's Guide: Radio Play, #5)", 'The Long Dark Tea-Time of the Soul (Dirk Gently, #2)', "Dirk Gently's Holistic Detective Agency (Dirk Gently, #1)", 'The Salmon of Doubt (Dirk Gently, #3)', "Mostly Harmless (Hitchhiker's Guide to the Galaxy, #5)", "Life, the Universe and Everything (Hitchhiker's Guide, #3)", 'The Long 

To find the longest book title among the books of the top 20 authors regarding their average rating

In [88]:
# Get the top 20 authors sorted by average_rating
top_20_authors = df_lighter_books.sort(col("average_rating").desc()).limit(20)

# Add a column with the length of each title
top_20_authors = top_20_authors.withColumn("title_length", length(col("title")))

# Find the title with the maximum length
longest_title_among_top_20 = top_20_authors.sort(col("title_length").desc()).select("title").first()[0]

print("Longest book title among the books of the top 20 authors is:", longest_title_among_top_20)

Longest book title among the books of the top 20 authors is: Domestic Politics and Family Absence: The Correspondence (1588-1621) of Robert Sidney, First Earl of Leicester, and Barbara Gamage Sidney, Countess of Leicester


To find the longest book title overall

In [89]:
# Add a column with the length of each title
longest_title = df_lighter_books.withColumn("title_length", length(col("title")))

# Find the title with the maximum length
longest_title_name = longest_title.sort(col("title_length").desc()).select("title").first()[0]

print("Longest book title overall is:", longest_title_name)

Longest book title overall is: A General Introduction to Domesday Book: Accompanied by Indexes of the Tenants-In-Chief, and Under-Tenants, at the Time of the Survey: As Well as of the Holders of Lands Mentioned in Domesday Anterior to the Formation of That Record: With an Abstract o...


Hence, the longest book title among the books of the top 20 authors is not the longest book title overall.

In [90]:
# Find the title with the minimum length
shortest_title_name = longest_title.sort(col("title_length").asc()).select("title").first()[0]

print("Shortest book title overall is:", shortest_title_name)

Shortest book title overall is: a


The shortest title being "a" might be a bit unusual, but it's not impossible. There are indeed books with very short titles, including one-letter titles. However, if we suspect that this might be due to an error in data entry or processing, it could be worth investigating further. But for now, we will assume that the data is correct.